In [ ]:
import os
import nltk
import pandas as pd
import numpy as np
curentdirectory='/content/drive/MyDrive/Distilbert_model/'
os.chdir(curentdirectory)

In [ ]:
# !unzip wordnet.zip
# !unzip stopwords.zip
!pip install torch==1.7.1
!pip install transformers==3.5.1

     |████████████████████████████████| 776.8MB 20kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
     |████████████████████████████████| 1.3MB 8.3MB/s 
     |████████████████████████████████| 2.9MB 30.8MB/s 
     |████████████████████████████████| 901kB 41.0MB/s 
     |████████████████████████████████| 1.1MB 33.2MB/s 


In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
import keras
from tqdm import tqdm
import pickle
from keras.models import Model
import keras.backend as K
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
import itertools
from keras.models import load_model
from sklearn.utils import shuffle
import torch
from transformers import DistilBertTokenizer, TFDistilBertModel, DistilBertConfig
from transformers import TFDistilBertForSequenceClassification
import torch.nn.functional as nnf

In [ ]:
DEVICE = "cuda"
MAX_LEN = 64
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 10 
BERT_PATH = "distilbert-base-uncased"
MODEL_PATH = "DistilBert_modelh.h5"
TRAINING_FILE = "Cleaned_Training_Testing_Mapping_Files/train_iris_fullcleaned.xlsx"
TOKENIZER = DistilBertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)

In [ ]:
df=pd.read_excel(TRAINING_FILE)
num_classes=len(df.intent1.unique())

df=pd.read_excel('Cleaned_Training_Testing_Mapping_Files/train_iris_fullcleaned.xlsx')
num_classes=len(df.intent1.unique())
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)
MODEL = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=num_classes)
MODEL.compile(loss=loss,optimizer=optimizer, metrics=[metric])
MODEL.load_weights(MODEL_PATH)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
import time
import torch.nn as nn
import pandas as pd
tokenizer = TOKENIZER
max_len = MAX_LEN
sent_csv=pd.read_excel('Cleaned_Training_Testing_Mapping_Files/test_iris_fullcleaned.xlsx')
sentences=sent_csv.utterance.values
d= dict([(i,a) for i, a in zip(df.intent1, df.intent)])
maxindx,predictions,maxindx1,maxindx2,maxindx3,maxindx4,predictions1,predictions2,predictions3,predictions4=[],[],[],[],[],[],[],[],[],[]
top_p1,top_p2,top_p3,top_class1,top_class2,top_class3=[],[],[],[],[],[]
for sent in sentences:
    DistilBert_inp=tokenizer.encode_plus(sent,add_special_tokens = True,max_length =64,pad_to_max_length = True,return_attention_mask = True)
    input_ids=DistilBert_inp['input_ids']
    attention_masks=DistilBert_inp['attention_mask']
    input_ids=np.asarray(input_ids).reshape(-1,64)
    attention_masks=np.array(attention_masks).reshape(-1,64)
    outputs = MODEL.predict([input_ids,attention_masks],batch_size=32)
    maxidx = outputs[0].argmax(axis=1)
    maxindx.append(int(maxidx))
    print(d[int(maxidx)])

    predictions.append(d[int(maxidx)])
    # outputs = torch.sigmoid(outputs).cpu().detach().numpy()
    print(str(outputs[0]))
    
    prob = nnf.softmax(torch.tensor(outputs[0]),dim=1)
    top_p, top_class = prob.topk(3, dim = 1)
    print("confidence score",top_p)
    print("confidence score1",top_p[0][0])
    
    top_p1.append(float(top_p[0][0]))
    top_p2.append(float(top_p[0][1]))
    top_p3.append(float(top_p[0][2]))
    print("top_class",top_class)
    print("top_class1",top_class[0][0])

    top_class1.append(int(top_class[0][0]))
    top_class2.append(int(top_class[0][1]))
    top_class3.append(int(top_class[0][2]))
    
    predictions1.append(d[int(top_class[0][0])])
    predictions2.append(d[int(top_class[0][1])])
    predictions3.append(d[int(top_class[0][2])])

sent_csv["pred_idx"]=pd.DataFrame(maxindx)
sent_csv["pred_values"]=pd.DataFrame(predictions)
sent_csv["top_p1"]=pd.DataFrame(top_p1)
sent_csv["top_p2"]=pd.DataFrame(top_p2)
sent_csv["top_p3"]=pd.DataFrame(top_p3)
sent_csv["top_class1"]=pd.DataFrame(top_class1)
sent_csv["top_class2"]=pd.DataFrame(top_class2)
sent_csv["top_class3"]=pd.DataFrame(top_class3)
sent_csv["predictions1"]=pd.DataFrame(predictions1)
sent_csv["predictions2"]=pd.DataFrame(predictions2)
sent_csv["predictions3"]=pd.DataFrame(predictions3)
sent_csv.to_csv('IRIS_predictions.csv',index=None)
from sklearn.metrics import accuracy_score
accuracy_score(sent_csv['intent1'],sent_csv['pred_idx'])